# Automatizador de planilhas

In [1]:
import os
import shutil
import pandas as pd
import numpy as np
from openpyxl import Workbook, load_workbook

In [2]:
def cpf_formatado(cpf):
    cpf_list = list(cpf)  
    if len(cpf_list) < 11:
        cpf = cpf.zfill(11)
    cpf = '{}.{}.{}-{}'.format(cpf[:3], cpf[3:6], cpf[6:9], cpf[9:])
    
    return(cpf)

In [ ]:
def arquivos(nome, fabricante_modulo, modulo_ref, fabricante_inversor, modelo_inversor):
    os.makedirs(f"arquivos - {nome}/") 
    dst = f"arquivos - {nome}/" 
    
    #---------------DATASHEET-PAINEL---------
    src_datasheet_painel = f"paineis/{fabricante_modulo}/datasheet/{modulo_ref}.pdf"
    shutil.copy2 (src_datasheet_painel, dst)
    os.rename(f"arquivos - {nome}/{modulo_ref}.pdf", f"arquivos - {nome}/DATASHEET MÓDULO.pdf")
    
    
    #---------------PRINT-PAINEL-------------
    src_print_painel = f"paineis/{fabricante_modulo}/prints/{modulo_ref}.png"
    shutil.copy2 (src_print_painel, dst)
    
    #---------------DATASHEET-INVERSOR---------
    src_datasheet_inversor = f"inversores/{fabricante_inversor}/datasheet/{modelo_inversor}.pdf"
    shutil.copy2 (src_datasheet_inversor, dst)
    os.rename(f"arquivos - {nome}/{modelo_inversor}.pdf", f"arquivos - {nome}/DATASHEET INVERSOR.pdf")

    
    #---------------INMETRO-INVERSOR---------
    src_inmetro_inversor = f"inversores/{fabricante_inversor}/inmetro/{modelo_inversor}.pdf"
    shutil.copy2 (src_inmetro_inversor, dst)
    os.rename(f"arquivos - {nome}/{modelo_inversor}.pdf", f"arquivos - {nome}/INMETRO INVERSOR.pdf")
    
    #---------------PRINT-INVERSOR---------
    src_print_inversor = f"inversores/{fabricante_inversor}/prints/{modelo_inversor}.png"
    shutil.copy2 (src_print_inversor, dst)
    
    
    return("arquivos do inversor copiados")

In [30]:
nome = 'TANIA REGINA CALDAS DE CASTRO'
fabricante = 'SOLPLANET'
modelo = 'ASW4000-S'

arquivos(nome, fabricante, modelo)

In [4]:
def carga_declarada(disjuntor):
    if disjuntor == 25:
        CD = 5
    elif disjuntor == 30 or disjuntor == 32:
        CD = 6
    elif disjuntor == 40:
        CD = 8
    elif disjuntor == 50:
        CD = 10
    elif disjuntor == 60 or disjuntor == 63:
        CD = 12
    
    return(CD)

In [5]:
def info_cliente(cod, path="planilhas/tabela_cliente.xlsx"):
    
    df = pd.DataFrame(pd.read_excel(path))
    df = df.astype('object') 
    df_filtrado = df[(df['CÓDIGO']==cod)]
    dict_infos = df_filtrado.to_dict(orient='index')
    
    ## Essa laço for é necessária devido a 'key' do dicionário gerado pela função
    ## 'to_dict' ser um valor inteiro determinado pela posição da linha, logo 
    ## cada cliente terá uma 'key' diferente. 
    global i
    
    for i in dict_infos: 
        a = i
    
    dict_infos = dict_infos.get(i)
    
    for key in dict_infos:
        if isinstance(dict_infos[key], str) == False:
            dict_infos[key] = str(dict_infos[key])

        dict_infos['CÓDIGO'] = int(dict_infos['CÓDIGO'])
        dict_infos['DISJUNTOR DE ENTRADA'] = int(dict_infos['DISJUNTOR DE ENTRADA'])
        dict_infos['QTD MOD'] = int(dict_infos['QTD MOD'])
        
    return (dict_infos)

In [6]:
def modulo(modulo, path="planilhas/placas.xlsx"):
    df_m = pd.DataFrame(pd.read_excel(path))
    df_modulo = df_m[df_m['NOME REF'] == modulo]
    dict_modulo = df_modulo.to_dict(orient='index')
    
    for k in dict_modulo:
        j = k
        
    
    dict_modulo = dict_modulo.get(j)  
    
    return(dict_modulo)

In [7]:
def inversor(inversor, path="planilhas/inversores.xlsx"):
    df_i = pd.DataFrame(pd.read_excel(path))
    df_inversor = df_i[df_i['MODELO'] == inversor]
    dict_inversor = df_inversor.to_dict(orient='index')
    
    
    for l in dict_inversor:
        m = l
        
    
    dict_inversor = dict_inversor.get(m)  
    
    return(dict_inversor)

In [8]:
def comissionamento(infos, path="planilhas/relatorio_comissionamento.xlsx"):
    wb = load_workbook(path)
    ws = wb.active
    
    ws['C6'] = infos.get('NOME')
    ws['O6'] = infos.get('CONTA CONTRATO')
    ws['C9'] = infos.get('CEP')
    ws['F9'] = infos.get('MUNICÍPIO')
    ws['C8'] = ','.join(infos.get('ENDEREÇO').split(',')[0:-1])
    ws['N8'] = infos.get('ENDEREÇO').split(',')[-1].lstrip()    
    
    primeiro_nome = infos.get('NOME').split(' ')[0].lower()
    wb.save(f'arquivos - {primeiro_nome}/relatorio_comissionamento_{primeiro_nome}.xlsx')
    
    wb.close()

In [9]:
def vistoria(infos, path="planilhas/Anexo-VI-Formulario-de-Solicitacao-de-Vistoria-de-Microgeracao-Distribuida.xlsx"):
    wb = load_workbook(path)
    ws = wb.active
    
    ws['C12'] = infos.get('NOME')
    ws['R12'] = cpf_formatado(infos.get('CPF'))
    ws['AC11'] = infos.get('RG')
    ws['C17'] = infos.get('ENDEREÇO')
    ws['I17'] = infos.get('MUNICÍPIO')
    ws['D17'] = infos.get('CEP')
    ws['G19'] = infos.get('RAMO DE ATIVIDADE')
    ws['X19'] = infos.get('CLASSE')
    ws['O21'] = infos.get('TIPO DE LIGAÇÃO')
    ws['H13'] = infos.get('DISJUNTOR DE ENTRADA')
    ws['F21'] = infos.get('TIPO DE RAMAL')
    ws['P25'] = infos.get('X')
    ws['Y25'] = infos.get('Y')
    ws['I43'] = infos.get('ENQUADRAMENTO')
    ws['W9'] = infos.get('CONTA CONTRATO')
    
    primeiro_nome = infos.get('NOME').split(' ')[0].lower()
    wb2 = load_workbook(f'arquivos - {primeiro_nome}/solicitacao_acesso_{primeiro_nome}.xlsx')
    ws2 = wb2['1']
    
    ws['AC43'] = ws2['F54']
    
    wb.save(f'arquivos - {primeiro_nome}/SOLICITAÇÃO DE VISTORIA - {primeiro_nome.upper()}.xlsx')
    wb.close()
    

In [10]:
def acesso(infos, path="planilhas/solicitacao_acesso.xlsx"):
    wb = load_workbook(path)
    ws0 = wb['0']
    ws1 = wb['1']
    
    ws1['C10'] = infos.get('NOME')
    ws1['R10'] = cpf_formatado(infos.get('CPF'))
    ws1['AC9'] = infos.get('RG')
    ws1['C13'] = infos.get('ENDEREÇO')
    ws1['I15'] = infos.get('MUNICÍPIO')
    ws1['D15'] = infos.get('CEP')
    ws1['G21'] = infos.get('RAMO DE ATIVIDADE')
    ws1['F25'] = infos.get('CLASSE')
    ws1['T25'] = infos.get('TIPO DE LIGAÇÃO')
    ws1['P27'] = infos.get('DISJUNTOR DE ENTRADA')
    ws1['F29'] = infos.get('TIPO DE RAMAL')
    ws1['P31'] = infos.get('X')
    ws1['Y31'] = infos.get('Y')
    ws1['I52'] = infos.get('ENQUADRAMENTO')
    ws1['AB19'] = infos.get('CONTA CONTRATO')
    ws1['F27'] = carga_declarada(infos.get('DISJUNTOR DE ENTRADA'))
    
    modulo_ref = infos.get('PAINEL') 
    infos_modulo = modulo(modulo_ref)
    
    inversor_ref = infos.get('INVERSOR')
    infos_inversor = inversor(inversor_ref)
    
    ws0['D7'] = infos_modulo.get('POTÊNCIA (W)')
    ws0['T7'] = infos_modulo.get('FABRICANTE')
    ws0['AA7'] = infos_modulo.get('MODELO')
    
    num_placas = float(infos.get('QTD MOD'))
    ws0['H7'] = num_placas
    ws0['P7'] = num_placas * 2.3
    
    ws0['D22'] = infos_inversor.get('MARCA')
    ws0['H22'] = infos_inversor.get('MODELO')
    ws0['L22'] = infos_inversor.get('POTÊNCIA NOMINAL (kW)')
    ws0['P22'] = infos_inversor.get('FAIXA DE TENSÃO (V)')
    ws0['T22'] = infos_inversor.get('CORRENTE NOMINAL (A)')
    ws0['W22'] = infos_inversor.get('FATOR DE POTÊNCIA')
    ws0['Z22'] = infos_inversor.get('RENDIMENTO (%)')
    ws0['AC22'] = infos_inversor.get('DHT')
    
    primeiro_nome = infos.get('NOME').split(' ')[0].lower()
    arquivos(primeiro_nome, infos_modulo.get('FABRICANTE'), modulo_ref, infos_inversor.get('MARCA'), infos_inversor.get('MODELO'))
        
    wb.save(f'arquivos - {primeiro_nome}/solicitacao_acesso_{primeiro_nome}.xlsx')
    wb.close()
    

In [11]:
def inf_memorial(i, infos, path="planilhas/TABELA_CLIENTES_AUX.xlsx"):
    wb = load_workbook(path)
    ws = wb.active
    
    s = i+2
    
    modulo_ref = infos.get('PAINEL') 
    infos_modulo = modulo(modulo_ref)
    
    inversor_ref = infos.get('INVERSOR')
    infos_inversor = inversor(inversor_ref)
    
    disjuntor = infos.get('DISJUNTOR DE ENTRADA')
    
    
    if infos.get('TIPO DE LIGAÇÃO') == 'MONOFÁSICO':
        num_fases = 1
        pot_nominal = 220
        num_fios = 2   
        fio = 10
    
    else:
        num_fases = 3
        pot_nominal = 360
        num_fios = 2
        
        if disjuntor == 40:
            fio = 10
        
        elif disjuntor == 60 or disjuntor == 63:
            fio = 16
        
        elif disjuntor == 80:
            fio = 25
        
        elif disjuntor == 100:
            fio = 35
            
        elif disjuntor == 125:
            fio = 50
            
            
    ##cálculos 
    PD_kva = (pot_nominal * disjuntor * num_fases) / 1000
    PD_kva = round(PD_kva, 2)
    PD_kw = PD_kva * 0.92
    PD_kw = round(PD_kw, 2)
    POT_kwp = (infos_modulo.get('POTÊNCIA (W)') * infos.get('QTD MOD'))/1000
    POT_kwp = round(POT_kwp, 2)
               
    ws[f'V{s}'] = POT_kwp
    ws[f'W{s}'] = pot_nominal
    ws[f'Z{s}'] = num_fios
    ws[f'AA{s}'] = num_fases  
    ws[f'X{s}'] = infos_modulo.get('POTÊNCIA (W)')
    ws[f'Y{s}'] = infos_inversor.get('POTÊNCIA NOMINAL (kW)')
    ws[f'AB{s}'] = infos_modulo.get('TIPO')
    ws[f'AC{s}'] = infos_modulo.get('MODELO')
    ws[f'AD{s}'] = PD_kva  
    ws[f'AE{s}'] = PD_kw
    ws[f'AF{s}'] = fio
    ws[f'AG{s}'] = infos_inversor.get('DISJUNT')
    
    wb.save('planilhas/TABELA_CLIENTES_AUX.xlsx')
    wb.close()

In [12]:
def preencher_acesso():
    cod = int(input('Código do cliente: '))
    num_placas = int(input('Quantidade de placas: '))
    infos = info_cliente(cod)
    
    acesso(infos, num_placas)

In [13]:
def main():
    
    cod = int(input('Código do cliente: '))
    #num_placas = int(input('Quantidade de placas: '))
    infos = info_cliente(cod)
    
    acesso(infos)
    comissionamento(infos)
    
    inf_memorial(i, infos) 
    #vistoria(infos)

In [15]:
main()

Código do cliente:  70
